In [1]:
from awpy import Demo

import pandas as pd
import numpy as np

import seaborn as sns

import random
import sys
import os

pd.set_option('display.max_columns', 100)

sys.path.append(os.path.abspath('./package'))

In [2]:
dem = Demo('../../data/matches-raw/cs2/2023/_iem-sydney-2023-apeks-vs-gamerlegion-bo3-92jrigmL2p42QxNJ6dsNw2apeks-vs-gamerlegion-m1-vertigo.dem')

In [10]:
dem.events.keys()

dict_keys(['round_freeze_end', 'player_sound', 'hegrenade_detonate', 'player_connect_full', 'item_pickup', 'smokegrenade_expired', 'item_pickup_slerp', 'round_announce_match_point', 'cs_round_start_beep', 'player_ping_stop', 'player_given_c4', 'round_officially_ended', 'hltv_versioninfo', 'flashbang_detonate', 'player_death', 'player_hurt', 'weapon_fire', 'bomb_planted', 'bomb_pickup', 'cs_win_panel_round', 'announce_phase_end', 'round_time_warning', 'entity_killed', 'player_team', 'cs_intermission', 'round_announce_match_start', 'vote_cast', 'bomb_defused', 'server_cvar', 'bomb_exploded', 'cs_win_panel_match', 'round_end_upload_stats', 'inferno_startburn', 'player_disconnect', 'inferno_expire', 'round_announce_last_round_half', 'bomb_dropped', 'decoy_detonate', 'cs_pre_restart', 'round_mvp', 'decoy_started', 'hltv_fixed', 'hltv_chase', 'round_end', 'smokegrenade_detonate', 'round_start', 'player_spawn', 'player_ping', 'cs_round_final_beep'])